Calculate current from voltage and line parameters

In [4]:
import numpy as np

bdata = np.genfromtxt("System_data/bdata.csv", delimiter=",")


In [7]:
vr = np.genfromtxt("dataset_pf_corrected/V_real.csv", delimiter=",")
vi = np.genfromtxt("dataset_pf_corrected/V_imag.csv", delimiter=",")

cfr = np.genfromtxt("dataset_pf_corrected/CF_real.csv", delimiter=",")
cfi = np.genfromtxt("dataset_pf_corrected/CF_imag.csv", delimiter=",")

ctr = np.genfromtxt("dataset_pf_corrected/CT_real.csv", delimiter=",")
cti = np.genfromtxt("dataset_pf_corrected/CT_imag.csv", delimiter=",")


In [20]:
cfr_vr_matrix  = np.zeros((vr.shape[1], cfr.shape[1]))
cfr_vi_matrix  = np.zeros((vr.shape[1], cfr.shape[1]))

cfi_vr_matrix  = np.zeros((vr.shape[1], cfr.shape[1]))
cfi_vi_matrix  = np.zeros((vr.shape[1], cfr.shape[1]))

ctr_vr_matrix  = np.zeros((vr.shape[1], cfr.shape[1]))
ctr_vi_matrix  = np.zeros((vr.shape[1], cfr.shape[1]))

cti_vr_matrix  = np.zeros((vr.shape[1], cfr.shape[1]))
cti_vi_matrix  = np.zeros((vr.shape[1], cfr.shape[1]))

for idx,branch in enumerate(bdata):
    fbus = int(branch[0])-1
    tbus = int(branch[1])-1
    R = branch[2]
    X = branch[3]
    B = branch[4]
    TAU = branch[5]

    AR = R/(TAU**2 *(R**2 + X**2))
    AI = B/(2*TAU**2) - X/(TAU**2 *(R**2 + X**2))

    BR = -R/(TAU*(R**2 + X**2))
    BI = X/(TAU*(R**2 + X**2))

    DR = R/(R**2 + X**2)
    DI = B/2 - X/(R**2 + X**2)


    cfr_vr_matrix[fbus,idx] = AR
    cfr_vr_matrix[tbus,idx] = BR

    cfr_vi_matrix[fbus,idx] = -AI
    cfr_vi_matrix[tbus,idx] = -BI

    cfi_vr_matrix[fbus,idx] = AI
    cfi_vr_matrix[tbus,idx] = BI

    cfi_vi_matrix[fbus,idx] = AR
    cfi_vi_matrix[tbus,idx] = BR



    ctr_vr_matrix[fbus,idx] = BR
    ctr_vr_matrix[tbus,idx] = DR

    ctr_vi_matrix[fbus,idx] = -BI
    ctr_vi_matrix[tbus,idx] = -DI

    cti_vr_matrix[fbus,idx] = BI
    cti_vr_matrix[tbus,idx] = DI

    cti_vi_matrix[fbus,idx] = BR
    cti_vi_matrix[tbus,idx] = DR


In [25]:
cfr_vr_matrix.T.shape

(186, 118)

In [32]:
vr_q = vr[0,:]
vi_q = vi[0,:]

cfr_a = cfr[0,:]
cfi_a = cfi[0,:]

ctr_a = ctr[0,:]
cti_a = cti[0,:]

ctr_hat = np.matmul(vr_q.T, ctr_vr_matrix) + np.matmul(vi_q.T, ctr_vi_matrix)
print(np.mean(ctr_a - ctr_hat))

-9.755779386991208e-16
